I still want to implement acrpo.

Ground nominal term orderings.

Equivariance means that X = Y can't be oriented. or add(X,Y) = add(Y,X)
But we're ground so we're probably better behaved

Rudi noted that slots are lot of like unification vars that never get narrowed and have a disjointness constraint.
https://egraphs.zulipchat.com/#narrow/channel/328972-general/topic/EGRAPHS.20Meeting.202025-03-20.3A.20Slotted.20E-Graphs/near/543056830

https://memoryleak47.github.io/slotted-egraphs-demystified-i/
https://medium.com/@mineralsteins/ultimate-code-compression-with-slotted-egraph-74f1859c9df5

So we need an unfailing variant of knuth bendix

renamings aren't bijective. They are 1-1.

https://www.cs.ru.nl/~cynthiakop/2024_isr/day2handout.pdf  Termination:
the higher-order recursive path ordering - Kop


In [ ]:
from dataclasses import dataclass, field

counter = 0
def fresh_counter() -> int:
    global counter
    counter += 1
    return counter

@dataclass(frozen=True)
class Slot():
    name : int = field(default_factory=fresh_counter)

@dataclass(frozen=True)
class App():
    f : str
    args : tuple["Term", ...] = ()

type Term = App | Slot
